In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 

df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head(10)

Primary processing: check for null values

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
print("perccentage of null value ",round((df['bmi'].isnull().sum()/df.shape[0])*100,2),"%")

As it is about 4 percent.We can drop it.

In [ ]:
df= df.dropna()

df.isnull().sum()

We are good to go!

In [ ]:
df.info()

In [ ]:
df_valid = df.copy() # saved a copy of the dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder

def categorical_transform(list_colm):
    le = LabelEncoder()
    for colm in list_colm:
        df[colm] = le.fit_transform(df[colm])
    
    return df.head(10)

list_colm = ['gender','ever_married','work_type','Residence_type','smoking_status']

categorical_transform(list_colm)

In [ ]:
df_valid_num = df.copy() #transformed all categorical colm

Check your Data is uniform ?

Vizualization plot : dist plot

In [ ]:
df = df.drop(columns=['id'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.figure(figsize=(20,25),facecolor='white')
plotnumber=1


for column in df:
    if plotnumber<11:
        ax = plt.subplot(5,5,plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
    plotnumber += 1
plt.show()

Check for the outliers:

vizualization: box plot

In [ ]:
fig,ax = plt.subplots(figsize=(20,10))

sns.boxplot(data = df[['work_type','avg_glucose_level','bmi']], width = 0.5, ax = ax, fliersize = 3)

In [ ]:
def remove_outlier(list_col):
    for col in list_col:
        q1 = df[col].quantile(.25)
        q3 = df[col].quantile(.75)
        iqr = q3-q1
        col_threshold=q3+(iqr*1.5) #45,160
        df[col].values[df[col].values > col_threshold] = col_threshold
    return df

list_col = ['bmi','avg_glucose_level']
remove_outlier(list_col)

In [ ]:
plt.figure(figsize=(20,25),facecolor='white')
plotnumber=1


for column in df:
    if plotnumber<11:
        ax = plt.subplot(5,5,plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
    plotnumber += 1
plt.show()

Now the data seems to be distributed. See the outlieres!

In [ ]:
fig,ax = plt.subplots(figsize=(20,10))

sns.boxplot(data = df[['work_type','avg_glucose_level','bmi']], width = 0.5, ax = ax, fliersize = 3)

In [ ]:
df.shape

Well we have saved the data points  by the outliers values with a threshold !

In [ ]:
data_no_outlier = df.copy()

Check how many columns are participating in label prediction.

    Corr Matrix
    VIF
    PCA

In [ ]:
X = data_no_outlier.drop(columns = ['stroke'])
y = data_no_outlier['stroke']

In [ ]:
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1
for column in X:
    if plotnumber<=9 :
        ax = plt.subplot(3,3,plotnumber)
        sns.stripplot(y,X[column])
    plotnumber+=1
plt.tight_layout()

In [ ]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
X_scaled = scalar.fit_transform(X)


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["vif"] = [variance_inflation_factor(X_scaled,i) for i in range(X_scaled.shape[1])]
vif["Features"] = X.columns

In [ ]:
vif

We can consider all of those columns.

In [ ]:
corr = df.corr()

corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

principalComponents = pca.fit_transform(X)
explainedVarianceRatio = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
plt.figure(figsize=(10,8),facecolor='grey')
plt.plot(explainedVarianceRatio)
plt.xlabel('no of pca componenents')
plt.ylabel('Explained Variance Ratio')

In [ ]:

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score

#logistic regression as bench mark

from sklearn.linear_model import LogisticRegression


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_scaled,y, test_size= 0.25, random_state=35)

In [ ]:
logistic_reg = LogisticRegression()

In [ ]:
logistic_reg.fit(x_train,y_train)

In [ ]:
y_pred = logistic_reg.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat

In [ ]:
true_positive = conf_mat[0][0]
false_positive = conf_mat[0][1]
false_negative = conf_mat[1][0]
true_negative = conf_mat[1][1]
# Breaking down the formula for Accuracy
Accuracy = (true_positive + true_negative) / (true_positive +false_positive + false_negative)
Accuracy


In [ ]:
Precision = true_positive/(true_positive+false_positive)
Precision

In [ ]:
Recall = true_positive/(true_positive+false_negative)
Recall

In [ ]:
F1_Score = 2*(Recall * Precision) / (Recall + Precision)
F1_Score

In [ ]:
auc = roc_auc_score(y_test, y_pred)
auc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

In [ ]:
plt.plot(fpr, tpr, color='orange', label='ROC')
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--',label='ROC curve (area = %0.2f)' % auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()